In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score
import numpy as np

# Replace with the actual path to your CSV file
csv_file_path = 'path_to_your_csv.csv'

# Load the data from the CSV file
df = pd.read_csv(csv_file_path)

# Function to convert logits to probabilities using sigmoid
def logits_to_probabilities(logits):
    return 1 / (1 + np.exp(-logits))

# Applying sigmoid function to convert logits to probabilities
df['dMDetector_progan_prob'] = logits_to_probabilities(df['dMDetectorResultsinferenceResults_logits_Grag2021_progan'])
df['dMDetector_latent_prob'] = logits_to_probabilities(df['dMDetectorResultsinferenceResults_logits_Grag2021_latent'])
df['gANDetector_progan_prob'] = logits_to_probabilities(df['gANDetectorResultsinferenceResults_logits_gandetection_resnet50nodown_progan'])
df['gANDetector_stylegan2_prob'] = logits_to_probabilities(df['gANDetectorResultsinferenceResults_logits_gandetection_resnet50nodown_stylegan2'])

# Define the threshold for considering an image as synthetic
probability_threshold = 0.2

# Function to calculate the accuracy of each model based on the threshold
def calculate_accuracy_with_threshold(df, threshold):
    accuracies = {}
    for model in ['dMDetector_progan_prob', 'dMDetector_latent_prob', 'gANDetector_progan_prob', 'gANDetector_stylegan2_prob', 'exifDetectorResultsinferenceResults_isSyntheticImage']:
        df[f'{model}_pred'] = df[model] > threshold
        accuracies[f'{model}_accuracy'] = accuracy_score(df['true_label'], df[f'{model}_pred'])

    # Aggregate results: if any model predicts synthetic, consider the final prediction as synthetic
    prediction_columns = [f'{model}_pred' for model in ['dMDetector_progan_prob', 'dMDetector_latent_prob', 'gANDetector_progan_prob', 'gANDetector_stylegan2_prob', 'exifDetectorResultsinferenceResults_isSyntheticImage']]
    df['final_pred'] = df[prediction_columns].any(axis=1)
    accuracies['aggregate_accuracy'] = accuracy_score(df['true_label'], df['final_pred'])

    return accuracies

# Calculate accuracy
accuracies = calculate_accuracy_with_threshold(df, probability_threshold)
print(accuracies)
